## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

In [1]:
pip install cassandra-driver


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Import Packages 

In [2]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [3]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

from cassandra.cluster import Cluster
contact_points = ['127.0.0.1']
port = 9042

cluster = Cluster(contact_points, port=port)
session = cluster.connect()

#### Create & Set Keyspace

In [4]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above

# Create a keyspace
keyspace_query = """
CREATE KEYSPACE IF NOT EXISTS soundcloud
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
"""
session.execute(keyspace_query)

In [5]:
session.set_keyspace('soundcloud')

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [6]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command                      

query1_create_table = """
CREATE TABLE IF NOT EXISTS song_by_session_item (
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    song_length float,
    PRIMARY KEY (session_number, item_in_session_number)
)
"""
session.execute(query1_create_table)

### Let's insert our data into of table

In [7]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file_name = 'event_data.csv'

# Insert data into the table for Query 1
with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header in the csv file
    for row in csv_reader:
        query = """
        INSERT INTO song_by_session_item (session_number, item_in_session_number, artist_name, song_title, song_length)
        VALUES (%s, %s, %s, %s, %s)
        """
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))



### Validate our Data Model using a SELECT

In [8]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

query1 = """
SELECT artist_name, song_title, song_length
FROM song_by_session_item
WHERE session_number = 338 AND item_in_session_number = 4
"""
rows_query1 = session.execute(query1)

# Display results for Query 1
for row in rows_query1:
    print("-------------------------------------------")
    print(f"Artist: {row.artist_name}")
    print(f"Song Title: {row.song_title}")
    print(f"Song Length: {row.song_length} seconds")
    print("-------------------------------------------")


-------------------------------------------
Artist: Faithless
Song Title: Music Matters (Mark Knight Dub)
Song Length: 495.30731201171875 seconds
-------------------------------------------


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [9]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command  

query2_create = """
CREATE TABLE IF NOT EXISTS song_by_user_session (
    user_id int,
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    user_first_name text,
    user_last_name text,
    PRIMARY KEY ((user_id, session_number), item_in_session_number)
)
"""
session.execute(query2_create)


### Let's insert our data into of table

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header
## Task: Write the INSERT statements and assign it to the query variable
    for row in csv_reader:
        query = """
        INSERT INTO song_by_user_session (user_id, session_number, item_in_session_number, artist_name, song_title, user_first_name, user_last_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [11]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

query2 = """
SELECT artist_name, song_title, user_first_name, user_last_name
FROM song_by_user_session
WHERE user_id = 10 AND session_number = 182
ORDER BY item_in_session_number
"""
rows_query2 = session.execute(query2)

# Display results for Query 2
for row in rows_query2:
    print("--------------------------------------------------")
    print(f"Artist: {row.artist_name}")
    print(f"Song Title: {row.song_title}")
    print(f"User: {row.user_first_name} {row.user_last_name}")
    print("--------------------------------------------------")

--------------------------------------------------
Artist: Down To The Bone
Song Title: Keep On Keepin' On
User: Sylvie Cruz
--------------------------------------------------
--------------------------------------------------
Artist: Three Drives
Song Title: Greece 2000
User: Sylvie Cruz
--------------------------------------------------
--------------------------------------------------
Artist: Sebastien Tellier
Song Title: Kilometer
User: Sylvie Cruz
--------------------------------------------------
--------------------------------------------------
Artist: Lonnie Gordon
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
User: Sylvie Cruz
--------------------------------------------------


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [12]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command      

query3_create = """
CREATE TABLE IF NOT EXISTS user_by_song (
    song_title text,
    user_first_name text,
    user_last_name text,
    PRIMARY KEY (song_title, user_first_name, user_last_name)
)
"""

session.execute(query3_create)

### Let's insert our data into of table

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = """
        INSERT INTO user_by_song (song_title, user_first_name, user_last_name)
        VALUES (%s, %s, %s)
        """
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [14]:
query3 = """
SELECT user_first_name, user_last_name
FROM user_by_song
WHERE song_title = 'All Hands Against His Own'
"""
rows_query3 = session.execute(query3)
print("Users who listened to the song 'All Hands Against His Own':")
for row in rows_query3:
    print(f"User Full Name: {row.user_first_name} {row.user_last_name}")

Users who listened to the song 'All Hands Against His Own':
User Full Name: Jacqueline Lynch
User Full Name: Sara Johnson
User Full Name: Tegan Levine


### Drop the tables before closing out the sessions

In [15]:
session.execute("drop table song_by_session_item ")

In [16]:
session.execute("drop table song_by_user_session ")

In [17]:
session.execute("drop table user_by_song ")

### Close the session and cluster connection¶

In [18]:
session.shutdown()

In [19]:
cluster.shutdown()